# 1.　ngrokのauth_tokeを設定して前回の内容を実行[事前準備、設定]

In [1]:
#@title #1.　ngrokのauth_tokeを設定して前回の内容を実行[事前準備、設定]

!pip install --upgrade pip #12/27 21:46追加

# 1,2 Flask, pyngrokのインストール
!pip install flask pyngrok
#  必要なモジュールのimport
import os
# flaskからFlaskクラスをインポート
from flask import Flask
# pyngrokからngrokモジュールとconfモジュールをインポート
from pyngrok import ngrok, conf
# 設定
# ngrokトークンを設定
conf.get_default().auth_token = "" #ここに自分のNgrokトークンを入れてください！！！！！


  Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.


## 1-2 Flask-SQLAlchemyのインストール

In [2]:
# @title 1-2 Flask-SQLAlchemyのインストール
!pip install --upgrade flask-sqlalchemy #12/27 21:46追加

  Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.


## 1-3 OpenAIライブラリのインストール

In [3]:
# @title #1-3 OpenAIライブラリのインストール
# https://platform.openai.com/docs/api-reference/introduction
!pip install openai==1.55.3 httpx==0.27.2 anyio==3.7.0 

  Using cached anyio-3.7.0-py3-none-any.whl.metadata (4.7 kB)
Using cached anyio-3.7.0-py3-none-any.whl (80 kB)
  Attempting uninstall: anyio
    Found existing installation: anyio 3.6.2
    Not uninstalling anyio at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'anyio'. No files were found to uninstall.


## 1-4.Email_validatorのインストール

In [4]:
!pip install email-validator
!pip freeze > requirements.txt

## 4.アプリケーションを初期化する

In [5]:
#@title　#4.アプリケーションを初期化する

from flask import g, render_template

# テンプレートと静的ファイルの置き場所を定数に代入
TEMPLATE_PATH = 'flask_blog/templates'
STATIC_PATH = 'flask_blog/static'
# Flaskのインスタンスを作るときにそれを利用してファイルパスを設定する
app = Flask(__name__, template_folder=TEMPLATE_PATH, static_folder=STATIC_PATH)

## 5.Flask-SQLAlcyemyをアプリに設定をする

In [6]:
#@title　#5.Flask-SQLAlcyemyをアプリに設定をする

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
  pass #これは何にもしないと言う意味

# SQLAlchemyをインスタンス化
db = SQLAlchemy(model_class=Base)

# SQLiteデータベースをappに設定する
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///sample_db.sqlite"
#  appを拡張とともに初期化
db.init_app(app)

## 10.ログイン機能を実装

### 10-1.LoginManager の設定

In [7]:
!pip install flask_login==0.6.3
!pip install flask-wtf
from flask_login import LoginManager

#インスタンス化
login_manager = LoginManager()
#アプリをログイン機能を紐付ける
login_manager.init_app(app)
#未ログインユーザーを転送する(ここでは'login'ビュー関数を指定)
login_manager.login_view = 'login'

#現在のログインユーザーの情報を保持し、必要なときに参照できるようになる。
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(user_id)

  Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.


## 6-1-1. Tagモデル, Post_Tagテーブル（中間テーブル）を作る

In [8]:
#@title　#7. Tagモデル, Blog_Tagテーブル（中間テーブル）を作る
from __future__ import annotations
from sqlalchemy import Table
from sqlalchemy import ForeignKey
from sqlalchemy import Column
from sqlalchemy.orm import relationship
from sqlalchemy import  DateTime, Integer, String
from sqlalchemy.orm import Mapped, mapped_column, validates

post_tag_table = Table(
    "post_tag",
    Base.metadata,
    Column("post_id", ForeignKey("post.id"), primary_key=True),
    Column("tag_id", ForeignKey("tag.id"), primary_key=True),
)

class Tag(db.Model):
    __tablename__ = "tag"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(nullable=False, unique=True)
    created_at: Mapped[int] = db.Column(db.DateTime(timezone=True))
    posts: Mapped[list[Post]] = relationship(
        "Post",
        secondary=post_tag_table,  # 正しい中間テーブルを指定
        back_populates="tags"
    )

    @validates("name")
    def validate_existance(self, key, value):
        if not value:
            raise ValueError(key)

        return value
    def to_json(self):
        return {
            "id": self.id,
            "name": self.name
        }



## 6-1-2. Addressモデル, User_Addressテーブル（中間テーブル）を作る

In [9]:
#@title　#7. Addressモデル, User_Addressテーブル（中間テーブル）を作る
from __future__ import annotations
from sqlalchemy import Table
from sqlalchemy import ForeignKey
from sqlalchemy import Column
from sqlalchemy.orm import relationship
from sqlalchemy import  DateTime, Integer, String
from sqlalchemy.orm import Mapped, mapped_column, validates

association_table = Table(
    "User_Address",
    Base.metadata,
    Column("user_id", ForeignKey("user.id")),
    Column("address_id", ForeignKey("address.id")),
)

class Address(db.Model):
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(nullable=False, unique=True)
    created_at: Mapped[int] = db.Column(db.DateTime(timezone=True))

    #ここにブログの関連
    users: Mapped[list[User]] = relationship(secondary=association_table)  #adress -> User

    @validates("name")
    def validate_existance(self, key, value):
        if not value:
            raise ValueError(key)

        return value
    def to_json(self):
        return {
            "id": self.id,
            "name": self.name
        }



## ⛄️6-4.  Userモデルを作る

In [10]:
# UserMixin、ログイン・ログアウトで必要なライブラリをインポート
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user
# パスワードをハッシュ化するライブラリをインポート
from werkzeug.security import check_password_hash, generate_password_hash

# UserMixinを継承
class User(db.Model, UserMixin):
  __tablename__ = 'user'
  id: Mapped[int] = mapped_column(primary_key=True)
  user_name: Mapped[str] = mapped_column(nullable=False)
  email: Mapped[str] = mapped_column(nullable=False)
  created_at: Mapped[int] = db.Column(db.DateTime(timezone=True))
  password_hash = db.Column(db.String(128))
  
  # パスワードチェックする関数を追加
  def check_password(self, password):
    return check_password_hash(self.password_hash, password)


In [11]:
# #@title　#6. Userモデルを作る
# class User(db.Model):
#     id: Mapped[int] = mapped_column(primary_key=True)
#     user_name: Mapped[str] = mapped_column(nullable=False)
#     mail:Mapped[str] = mapped_column(nullable=False)
#     pass_word:Mapped[str] = mapped_column(nullable=False)
#     created_at: Mapped[int] = db.Column(db.DateTime(timezone=True))

#     @validates("blog_id", "body", "user_name")
#     def validate_existance(self, key, value):
#         if not value:
#             raise ValueError(key)

#         return value

## 6-2. Postモデルを作る

In [12]:
#@title　#6. Postモデルを作る

class Post(db.Model):
    __tablename__ = "post"
    id: Mapped[int] = mapped_column(primary_key=True)
    user_id: Mapped[int]  = mapped_column(db.ForeignKey(User.id))
    title: Mapped[str]  = mapped_column(nullable=False)
    body: Mapped[str] = mapped_column(nullable=False)
    user_name: Mapped[str] = mapped_column(nullable=False)
    created_at: Mapped[int] = db.Column(db.DateTime(timezone=True))
    address:Mapped[str] = mapped_column(nullable=False)
    memo:Mapped[str] = mapped_column(nullable=True)
    likes:Mapped[int] = mapped_column(nullable=False, default=0)
    private:Mapped[bool] = mapped_column(nullable=False, default=False)
    tags: Mapped[list[Tag]] = relationship(
        "Tag",
        secondary=post_tag_table,  # 正しい中間テーブルを指定
        back_populates="posts"
    )

    # https://docs.sqlalchemy.org/en/20/orm/mapped_attributes.html#simple-validators
    @validates("title", "body", "user_name")
    def validate_existance(self, key, value):
        if not value:
            raise ValueError(key)

        return value

## 7. モデルをもとにテーブルを作成する

In [13]:
#@title #7.モデルをもとにテーブルを作成する

# アプリケーションが実行されるコンテキスト内で行うという指定のwith
with app.app_context():
  # db.drop_all() # 今まで作成したテーブルを一度すべて消す。6でBlogモデルに変更を加えてない場合は不要
  #  テーブルを作成
  db.create_all()

## 8-1. タグテーブルにデータを入れる

In [14]:
# #@title #8. タグテーブルにデータを入れる

# from sqlalchemy import delete
# with app.app_context():
#     db.session.execute(db.delete(Tag)) # 同じデータが作られるのを防ぐために一度全部消す
#     for name in ["日記", "ガラスペン", "文房具"]:#なるほど
#         tag = Tag(name=name)
#         db.session.add(tag)
#         db.session.commit()

## 8-2. アドレステーブルにデータを入れる

In [15]:
# #@title #8. アドレステーブルにデータを入れる

# from sqlalchemy import delete
# with app.app_context():
#     db.session.execute(db.delete((Address))) # 同じデータが作られるのを防ぐために一度全部消す
#     for name in ["日記", "ガラスペン", "文房具"]:#なるほど
#         address = Address(name=name)
#         db.session.add(tag)
#         db.session.commit()

## 8-2. Postテーブルにデータを入れる

In [16]:
# #@title #8. アドレステーブルにデータを入れる

# from sqlalchemy import delete
# with app.app_context():
#     db.session.execute(db.delete((Address))) # 同じデータが作られるのを防ぐために一度全部消す
# 適切なデータを挿入する関数
def insert_sample_data(session: Session):
    # サンプルデータ
    posts = [
        {
            "user_id": 1,
            "title": "My First Blog Post",
            "body": "This is the body of my first blog post.",
            "user_name": "Alice",
            "created_at": datetime.utcnow(),
            "address": "123 Main St, Springfield",
            "memo": "First post, feeling excited!",
            "likes": 10,
            "private": False
        },
    ]

    # データベースに挿入
    for post_data in posts:
        post = Post(**post_data)  # Post クラスをインスタンス化
        session.add(post)

    # コミットしてデータを保存
    session.commit()
    print("Sample data inserted successfully.")

# insert_sample_data(db.session)

### 10-3-1.新規登録フォームの作成

In [17]:
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import DataRequired, Email, EqualTo, Length

class RegistrationForm(FlaskForm):
    user_name = StringField('ユーザー名', validators=[DataRequired(), Length(min=3, max=25)])
    email = StringField('メールアドレス', validators=[DataRequired(), Email()])
    password = PasswordField('パスワード', validators=[DataRequired(), Length(min=6)])
    confirm_password = PasswordField('パスワード(確認用)', validators=[
        DataRequired(), EqualTo('password', message='パスワードが一致しません')
    ])
    submit = SubmitField('登録')


### 10-3-2.ログイン用フォーム

In [18]:
# from flask_wtf import FlaskForm
# from wtforms import StringField, PasswordField, SubmitField
# from wtforms.validators import DataRequired, Email


class LoginForm(FlaskForm):
    email = StringField('Email', validators=[DataRequired(), Email(message='正しいメールアドレスを入力してください')])
    password = PasswordField('Password', validators=[DataRequired()])
    submit = SubmitField('ログイン')

### 10-4.ビュー関数

In [19]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        #フォーム入力したアドレスがDB内にあるか検索
        user = User.query.filter_by(email=form.email.data).first()
        if user is not None:
            #check_passwordはUserモデル内の関数
            if user.check_password(form.password.data):
                #ログイン処理。ログイン状態として扱われる。
                login_user(user)
                next = request.args.get('next')
                if next == None or not next[0] == '/':
                    next = url_for('post_index') #🪵post_indexに変更
                return redirect(next)
                
            else:
                flash('パスワードが一致しません')
        else:
            flash('入力されたユーザーは存在しません')

    return render_template('login.html', form=form)

    # "next"がない = 「通常どおり、ログイン状態でアクセスしようとした」　ので、ログイン後に本来リダイレクトすべきページに遷移する。
    # "next"がある = 「未ログインでログイン必須のページにアクセスしようとした」ので、そのユーザーがアクセスしようとしたページにリダイレクトさせる。
    # 上記の条件分岐は以下のようなコードとなる。  

    # next = request.args.get('next')
    # if next == None or not next[0] == '/':
    # next = url_for('user_maintenance')
    # return redirect(next)


### 10-5.新規ユーザー登録時に、パスワードのハッシュ化

In [20]:
  #値の参照
  @property
  def password(self):
    raise AttributeError('password is not a readable attribute')
  #値の設定
  @password.setter
  def password(self, password):
      self.password_hash = generate_password_hash(password)

  # モデルクラスの初期設定の関数も修正。
  #"password_hash"から"hash"を取り除き、passwordはハッシュ化されていない状態で渡されることを明示する。
  #setterでパスワードを受け取ってハッシュ化して、その値を"password_hash"に入れられデータベースに登録されるという流れ。
  def __init__(self, email, user_name, password):
    self.email = email
    self.user_name = user_name
    self.password = password
    
  #新規登録のビュー関数"register"も修正。
  #モデルクラスをインスタンス化する際の引数"password_hash"から"hash"を取り除く。
#   @app.route("/register", methods=["GET", "POST"])
#   def register():
#     form = RegistrationForm()
  
#     if form.validate_on_submit():
#       user = User(email=form.email.data, username=form.username.data, password=form.password.data)
#       db.session.add(user)
#       db.session.commit()

@app.route('/register', methods=['GET', 'POST'])
def register():
    form = RegistrationForm()  # フォームインスタンスを作成
    if form.validate_on_submit():  # フォームが送信され、バリデーションが成功した場合
        hashed_password = generate_password_hash(form.password.data)
        new_user = User(
            user_name=form.user_name.data,
            email=form.email.data,
            password_hash=hashed_password
        )
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))  # 登録後、ログインページへリダイレクト
    return render_template('register.html', form=form)



### 10-6.ログアウト機能


In [21]:
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required

#@login_required　を付けることで、未ログインユーザーはログインページに転送されるようになる。
@app.route('/logout')
@login_required
def logout():
    #現在のユーザーをログアウト状態にする
    logout_user()
    return redirect(url_for('login'))

### 10-7.ログインユーザーのみアクセス可能にする

In [22]:
#以下のコードでリダイレクトするビュー関数と、エラーメッセージを定義する
#ビュー関数"login"にリダイレクトするよう定義
login_manager.login_view = 'login'

# 未ログインユーザーにメッセージ表示
def localize_callback(*args, **kwarg):
    return 'このページにアクセスするには、ログインが必要です'
login_manager.localize_callback = localize_callback

## 9. アプリケーションでデータを読み出す+テンプレートにデータを渡すプログラミング

### 9-1. ルート/, GET /blogs 一覧画面（フォーム付き）, POST /blogs データ作成機能

In [23]:
#@title #9-1. ルート/, GET /blogs 一覧画面（フォーム付き）, POST /blogs データ作成機能 （前回と同じ）


from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from flask import g, render_template, redirect, url_for, request, flash, session

#セッションを設定する
app.secret_key = 'timestamp'
app.permanent_session_lifetime = timedelta(days=5)
# /（ルート）にアクセスしたら次に書く関数(home)を実行するという宣言
@app.route("/")
def home():
    return render_template('home.html')

# /posts にHTTPメソッドがGETでアクセスしたblog_index関数を実行する
@app.get("/posts")
def post_index():
   #データベースからデータを引き出す（オブジェクトのリストの形で帰ってくる）
    # https://msiz07-flask-docs-ja.readthedocs.io/ja/latest/patterns/sqlalchemy.html#declarative
    # データベースから投稿を取得
    posts = Post.query.all()
    tags = Tag.query.all()
    # テンプレートにblogs変数を渡す
    return render_template('all_post2_copy.html', posts = posts,tags =tags, post = Post())

# /posts にHTTPメソッドがPOSTでアクセスしたblog_create関数を実行する
@app.route("/new", methods=["GET", "POST"])
@login_required  # このデコレータを追加することでログイン必須に
def post_create():
    if request.method == "POST":
        # フォームの値を取得してバリデーション
        title = request.form.get("title")
        # user_id = request.form.get("user_id")
        user_id = current_user.id
        user_name = current_user.user_name
        # user_name = request.form.get("user_name")
        body = request.form.get("body")
        address = request.form.get("address")
        memo = request.form.get("memo")
        private = request.form.get("private")

        if not title or not user_name or not body:
            flash("タイトル、ユーザー名、本文は必須項目です。")
            return redirect(url_for("post_create"))

        # 投稿データを保存
        try:
            post = Post(
                title=title,
                user_name=user_name,#一時的に追加
                user_id=user_id, #一時的に追加
                body=body,
                address=address,
                memo=memo,
                private=private,
            )
            db.session.add(post)
            db.session.commit()
            flash("投稿が作成されました。")
            return redirect(url_for("post_index"))
        except Exception as e:
            flash(f"予期せぬエラー: {e}")
            return redirect(url_for("post_create"))

    # GETリクエスト時: フォーム画面を表示
    return render_template('new_post.html')

### 9-2 GET /blogs/:id 更新用フォーム、Post(気持ちはpatch) /blogs/:id 更新処理

In [24]:
# @title 9-2 GET /blogs/:id 更新用フォーム、Post(気持ちはpatch) /blogs/:id 更新処理

@app.get('/posts/<string:id>')
def get_post(id):
    # 指定のidのpostオブジェクトを取得する
    post = db.get_or_404(Post, id)
    # https://flask-sqlalchemy.readthedocs.io/en/3.1.x/queries/#queries-for-views
    return render_template('detail.html', post = post)

#🪵postの編集&再投稿ページ
@app.get('/posts/<string:id>/edit')
@login_required  # このデコレータを追加することでログイン必須に
def edit_post(id):
    # 指定のidのblogオブジェクトを取得する
    post = db.get_or_404(Post, id)
    # post.likes = post.likes + 1
    # print("いいね数は"+str(post.likes))
    # https://flask-sqlalchemy.readthedocs.io/en/3.1.x/queries/#queries-for-views
    return render_template('new_post.html', post = post)    

@app.post('/posts/<string:id>')
def update_post(id):
    # 指定のidのpostオブジェクトを取得する
    post = db.get_or_404(Post, id)

    # 更新処理の場合
    if request.form.get('_method') == 'PATCH':
        try:
            post.title = request.form["title"]
            post.user_name = request.form["user_name"]
            post.body = request.form["body"]
            post.address = request.form["address"]
            post.memo = request.form["memo"]
            post.created_at = datetime.utcnow()
            # post.private = request.form["private"]
            post.private = False
            post.likes = 0
            db.session.commit()
            flash("データを更新しました")
        except ValueError as ve:
            flash(f"入力が不足しています#{ve}")
        except Exception as e:
            flash(f"予期せぬエラー #{e}")
        # 編集画面にGETでリダイレクトする
        return redirect(url_for("get_post", id=id))
    # 削除リクエストの場合
    elif request.form.get('_method') == 'DELETE':
        db.session.delete(post)
        db.session.commit()
        # 一覧画面にGETでリダイレクトする
        return redirect(url_for("post_index"))

https://flask.palletsprojects.com/en/stable/patterns/javascript/

### 🪵9-2-2 いいねカウンターの実装 #1/2 りおちん

In [25]:
# 🪵1/6 いいねカウンターのルートを修正 
@app.route('/posts/<int:post_id>/like', methods=['POST'])
def like_post(post_id):
    post = Post.query.get(post_id)
    
    if post:
        post.likes += 1
        db.session.commit()  # データベースの更新
        print("いいね数は"+str(post.likes))
        return jsonify({'likes': post.likes}), 200
    else:
        return jsonify({'error': 'Post not found'}), 404


### 9-4 Post /blogs/:id/tag フォームで送られてきたタグをブログに紐づける

In [26]:
# @title #9-4 Post /blogs/:id/tag フォームで送られてきたタグをブログに紐づける
@app.post('/posts/<string:id>/tag')
def create_tag(id):

  post = db.get_or_404(Post, id)
  tag_name = request.form["name"]
  tag = Tag.query.filter_by(name=tag_name).first() # nameで登録済みのタグを探す
  if tag is None:
    tag = Tag(name = tag_name) # もしなければTagを作る
    db.session.add(tag)
    db.session.commit()
  post.tags.append(tag)  # blogにタグを紐づける
  db.session.commit()
  # 確認コード例 (SQLAlchemyの場合)
  print(post.tags)  # データベースから取得した後、正しいタグリストが含まれているか

  return redirect(url_for("get_post", id = post.id))

## 11.アプリケーションを立ち上げる

In [32]:
#@title #10.アプリケーションを立ち上げる

if __name__ == "__main__":
    public_url = ngrok.connect(2000)
    print(f"ngrok URL: {public_url}")
    app.run(port=2000)

t=2025-01-07T12:17:14+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040
ngrok URL: NgrokTunnel: "https://0deb-3-220-159-34.ngrok-free.app" -> "http://localhost:2000"
 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on http://127.0.0.1:2000
Press CTRL+C to quit
127.0.0.1 - - [07/Jan/2025 12:17:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 12:17:27] "GET /api/placeholder/32/32 HTTP/1.1" 404 -
127.0.0.1 - - [07/Jan/2025 12:17:27] "GET /static/css/home.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 12:17:28] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 12:17:28] "GET /static/js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 12:17:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Jan/2025 12:17:30] "GET /posts HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 12:17:31] "GET /api/placeholder/32/32 HTTP/1.1" 404 -
127.0.0.1 - - [07/Jan/2025 12:17:31] "GET /static/css/style.css HTTP/1.1" 304 -


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5456fa35-ea0f-4cee-9aed-01d78c91284d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>